## Imoprt LIbrary



In [ ]:
pip install -q tensorflow-recommenders

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_recommenders as tfrs


## Import data from CSV

In [ ]:
makanan = pd.read_csv("/content/Dataset_Nutrisi_Makanan.csv")
users = pd.read_csv("/content/data_user.csv")

In [ ]:
makanan= tf.data.Dataset.from_tensor_slices(dict(makanan))

In [ ]:
user= tf.data.Dataset.from_tensor_slices(dict(users))

In [ ]:
users = users.map(lambda x: {
    "Nama_Makanan": x["Nama_Makanan"],
    "user_id": x["user_id"],
})
makanan = makanan.map(lambda x: x["Nama_Makanan"])

AttributeError: ignored

In [ ]:
tf.random.set_seed(42)
shuffled = users.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

AttributeError: ignored

In [ ]:
All_Nama_Makanan = makanan.batch(1_000)
user_ids = users.batch(1_000_000).map(lambda x: x["user_id"])

Nama_Makanan_Unik= np.unique(np.concatenate(list(All_Nama_Makanan)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

Nama_Makanan_Unik[:10]

array([b'Ayam Panggang (1 Porsi - 85 g)', b'Babi Panggang (100 g)',
       b'Bakso Ayam (1 Porsi)', b'Bakso Sapi (100 g)',
       b'Beef Patty_Roti Daging (1 Porsi)',
       b'Bola Ikan Udang Wortel (1 Porsi)', b'Buah Pir (100 g)',
       b'Bubur Jagung (1 Porsi)', b'Bubur Kacang Hijau (1 Porsi)',
       b'Buncis (100 g)'], dtype=object)

##Modelling

In [ ]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32)
])

/usr/local/lib/python3.10/dist-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


ValueError: ignored

In [ ]:
makanan_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=Nama_Makanan_Unik, mask_token=None),
  tf.keras.layers.Embedding(len(Nama_Makanan_Unik) + 1, 32)
])

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics= tfrs.metrics.FactorizedTopK(candidates=makanan.batch(128).map(makanan_model))
)

#Recommended system

In [ ]:
from typing import Dict, Text
class MakananlensModel(tfrs.Model):

  def __init__(self, user_model, makanan_model):
    super().__init__()
    self.makanan_model: tf.keras.Model = makanan_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_makanan_embeddings = self.makanan_model(features["Nama_Makanan"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_makanan_embeddings)


In [ ]:
model = MakananlensModel(user_model, makanan_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3


InvalidArgumentError: ignored

In [ ]:
model.evaluate(cached_test, return_dict=True)

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((makanan.batch(100), makanan.batch(100).map(model.makanan_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["20"]))
print(f"Recommendations for user 20: {titles[0, :3]}")